### Trade Demo

#### Goal: 
- Load the trade data for the country `Canada`
- Launch a domain node for canada
- Login into the domain node
- Convert required rows to private tensors in the `Canada` trade data
- Upload `Canada's` trade on the domain node
- Create a Data Scientist User and assign a privacy budget

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

canada = pd.read_csv("../datasets/ca - feb 2021.csv")

/home/user/anaconda3/envs/pysyft/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Step 1: Load the dataset

We have trade data for the country, which has provided data from Feb 2021. They key colums are:

- Commodity Code: the official code of that type of good
- Reporter: the country claiming the import/export value
- Partner: the country being claimed about
- Trade Flow: the direction of the goods being reported about (imports, exports, etc)
- Trade Value (US$): the declared USD value of the good

Let's have a quick look at the top five rows of the dataset.

In [2]:
canada.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0


### Step 2: Spin up the Domain Node (if you haven't already)

SKIP THIS STEP IF YOU'VE ALREADY RUN IT!!!

As the main requirement of this demo is to perform analysis on the Canada's trade dataset. So, we need to spin up a domain node for Canada.

Assuming you have [Docker](https://www.docker.com/) installed and configured with >=8GB of RAM, navigate to PySyft/packages/hagrid and run the following commands in separate terminals (can be done at the same time):


```bash
# install hagrid cli tool
pip install -e .
```

```bash
hagrid launch Canada domain
```

<div class="alert alert-block alert-info">
    <b>Quick Tip:</b> Don't run this now, but later when you want to stop these nodes, you can simply run the same argument with the "stop" command. So from the PySyft/grid directory you would run. Note that these commands will delete the database by default. Add the flag "--keep_db=True" to keep the database around. Also note that simply killing the thread created by ./start is often insufficient to actually stop all nodes. Run the ./stop script instead. To stop the nodes listed above (and delete their databases) run:

```bash
hagrid land Canada
```
</div>

### Step 3: Login into the Domain as the Admin User

In [3]:
import syft as sy

# Let's login into the domain node
domain_node = sy.login(email="info@openmined.org", password="changethis", port=8081)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!


In [4]:
canada.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0


In [5]:
# For, simplicity we will upload the first 10000 rows of the dataset.
canada = canada[:10000]

### Step 4: Convert the dataset to private pointers

In [6]:
from syft.core.adp.entity import Entity

In [7]:
# The 'Partner' column i.e the countries to which the data is exported 
# is private, therefore let's create entities for each of the partner defined

entities = [Entity(name=partner) for partner in canada["Partner"].unique()]

In [ ]:
# Since the 'Trade Value' i.e. the value of the goods exported/imported is private field, 
# therefore let's convert it to a private tensor.

canada["Trade Value (US$)"] = sy.Tensor(canada[col]).private(0.01, 100000000, entities=entities)

### Step 5: Upload Canada's trade data on the domain

In [ ]:
# Great !! we're logged in. Let's upload the dataset.
# For, simplicity we will upload the first 10000 rows of the dataset.

domain_node.load_dataset(
    assets={"feb2020": canada},
    name="Canada Trade Data - First 10000 rows",
    description="""A collection of reports from Canada's statistics 
                    bureau about how much it thinks it imports and exports from other countries.""",
)

Cool !!! The dataset was successfully uploaded onto the domain.

In [8]:
# Now, let's check datasets available on the domain.
domain_node.datasets.pandas()

,data,name,description,id,tags
0,"[{'name': 'feb2020', 'id': 'a523e69c-c36e-427d...",Canada Trade Data,A collection of reports from Canada's statisti...,1192bcd5-e7dc-4312-9fff-8f3b5c06c3ae,[]
1,"[{'name': 'feb2020', 'id': 'f4956227-4869-4682...",Canada Trade Data,A collection of reports from Canada's statisti...,269043c0-9067-4557-9c6a-bfd834f1be2d,[]
2,"[{'name': 'feb2020', 'id': '440b9751-6b5f-48f0...",Canada Trade Data,A collection of reports from Canada's statisti...,3789e774-b4d5-4902-ad4d-7096acd16f0e,[]
3,"[{'name': 'feb2020', 'id': 'e167e117-0926-4f93...",Canada Trade Data,A collection of reports from Canada's statisti...,cc28df5f-1b4c-4725-9762-2fb60c0740db,[]
4,"[{'name': 'feb2020', 'id': '4c89473a-f7bf-45db...",Canada Trade Data,A collection of reports from Canada's statisti...,6a49b4ed-0028-45d1-ad89-52b34350d257,[]
5,"[{'name': 'feb2020', 'id': '70313e75-0e29-437e...",Canada Trade Data - First 5000 rows,A collection of reports from Canada's statisti...,524c9e2b-ad10-4e39-a772-dc3e997c526e,[]
6,"[{'name': 'feb2020', 'id': 'f5b07736-5cc2-4e5d...",Canada Trade Data - First 5000 rows,A collection of reports from Canada's statisti...,ff32d492-b3de-4e47-9af9-5377faef0bea,[]
7,"[{'name': 'feb2020', 'id': '1838e963-8eaa-443e...",Canada Trade Data - First 5000 rows,A collection of reports from Canada's statisti...,75bf69bc-1c57-4070-8378-651146e93580,[]
8,"[{'name': 'feb2020', 'id': 'bd94c0dc-52b7-43fb...",Canada Trade Data - First 10000 rows,A collection of reports from Canada's statisti...,069eedd0-5f5d-4446-beaa-d89480796ed9,[]
9,"[{'name': 'feb2020', 'id': '7ad73649-e700-4d47...",Canada Trade Data - First 10000 rows,A collection of reports from Canada's statisti...,1fb490ac-8cd5-4688-8643-6d33d5e5d4b5,[]


### Step 6: Create a Data Scientist User

Open http://localhost:8081, login is the root user (username: info@openmined.org, password:changethis), and create a user with the following attributes:

- Name: Sheldon Cooper
- Email: sheldon@caltech.edu
- Password: bazinga

In [10]:
# Or, you can create a user from the notebook itself.
# We will first create an accountant for the user. An Accountant is used to assign
# and keep track of the privacy budget for a given user.

from syft.core.adp.adversarial_accountant import AdversarialAccountant

# Create an Adversarial Accountant for the user and assign a privacy budget to the user
accountant = AdversarialAccountant(max_budget=100000)


# Now, let's create an account for the user.
domain_node.users.create(
    **{
        "name": "Sheldon Cooper",
        "email": "sheldon@caltech.edu",
        "password": "bazinga",
        "accountant": accountant,
    },
)


# Or
domain_node.users.create(
    **{
        "name": "Sheldon Cooper",
        "email": "sheldon@caltech.edu",
        "password": "bazinga",
    },
)
user = domain_node.users[0]
user.assign_budget(max_buget=100000)

```
Great !!! We were successfully able to create a new user and assign a privacy budget.
Now, let's move to the Data Scientist notebook, to check out their experience.
```

### Step 7: Decline request to download entire datsaet

In [ ]:
# Let's check if there are any requests pending for approval.
domain_node.requests

In [ ]:
# Looks like the DS wants to download the whole dataset. We cannot allow that.
# Let's select and deny this request.
domain_node.requests[0].deny()

### STOP: Return to Data Scientist - Canada.ipynb - STEP 3!!